This notebook runs the simulated image tests in Section 3 of the manuscript. It generates pairs of images with additive gaussian noise as outlined in Section 3.1, and uses both the Bramich 2008 (B08) and PyTorchDIA algorithms to infer the associated kernel and (scalar) differential background. For each difference image pair, we compute the model fit quality and photometric accuracy metrics in Section 3.2, and append each result to separate text files for the B08 and PyTorchDIA solutions respectively.

Firstly, we'll run a couple of cells required for the B08 algorithm. These are taken from the pyDANDIA pipeline, https://github.com/pyDANDIA/pyDANDIA.

In [1]:
%load_ext Cython

In [2]:
%%cython

from __future__ import division
import numpy as np
cimport numpy as np
cimport cython
DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

# compile suggestion: gcc -shared -pthread -fPIC -fwrapv -O2 -Wall -fno-strict-aliasing -I/somepath/include/python2.7 -o umatrix_routine.so umatrix_routine.c

@cython.boundscheck(False) # turn off bounds-checking
@cython.wraparound(False)  # turn off negative index wrapping
@cython.nonecheck(False)  # turn off negative index wrapping

def umatrix_construction(np.ndarray[DTYPE_t, ndim = 2] reference_image,np.ndarray[DTYPE_t, ndim = 2] weights, pandq, n_kernel_np, kernel_size_np):

    cdef int ni_image = np.shape(reference_image)[0]
    cdef int nj_image = np.shape(reference_image)[1]
    cdef double sum_acc = 0.
    cdef int idx_l,idx_m,idx_l_prime,idx_m_prime,idx_i,idx_j
    cdef int kernel_size = np.int(kernel_size_np)
    cdef int kernel_size_half = np.int(kernel_size_np)/2
    cdef int n_kernel = np.int(n_kernel_np)
    cdef np.ndarray u_matrix = np.zeros([n_kernel + 1, n_kernel + 1], dtype=DTYPE)

    for idx_p in range(n_kernel):
        for idx_q in range(idx_p,n_kernel):
            sum_acc = 0.
            idx_l, idx_m = pandq[idx_p]
            idx_l_prime, idx_m_prime = pandq[idx_q]
            for idx_i in range(kernel_size_half,ni_image-kernel_size+kernel_size_half+1):
                for idx_j in range(kernel_size_half,nj_image-kernel_size+kernel_size_half+1):
                    sum_acc += reference_image[idx_i + idx_l, idx_j + idx_m] * reference_image[idx_i + idx_l_prime,idx_j + idx_m_prime]  * weights[idx_i, idx_j]
            u_matrix[idx_p, idx_q] = sum_acc
            u_matrix[idx_q, idx_p] = sum_acc

    for idx_p in [n_kernel]:
        for idx_q in range(n_kernel):
            sum_acc = 0.
            idx_l = kernel_size
            idx_m = kernel_size
            idx_l_prime, idx_m_prime = pandq[idx_q]
            for idx_i in range(kernel_size_half,ni_image-kernel_size+kernel_size_half+1):
                for idx_j in range(kernel_size_half,nj_image-kernel_size+kernel_size_half+1):
                    sum_acc += reference_image[idx_i + idx_l_prime, idx_j + idx_m_prime] * weights[idx_i, idx_j]
            u_matrix[idx_p, idx_q] = sum_acc
    
    for idx_p in range(n_kernel):
        for idx_q in [n_kernel]:
            sum_acc = 0.
            idx_l, idx_m = pandq[idx_p]
            idx_l_prime = kernel_size
            idl_m_prime = kernel_size
            for idx_i in range(kernel_size_half,ni_image-kernel_size+kernel_size_half+1):
                for idx_j in range(kernel_size_half, nj_image-kernel_size+kernel_size_half+1):
                    sum_acc += reference_image[idx_i + idx_l, idx_j + idx_m] * weights[idx_i, idx_j] 
            u_matrix[idx_p, idx_q] = sum_acc

    sum_acc = 0.
    for idx_i in range(ni_image):
        for idx_j in range(nj_image):
            sum_acc += weights[idx_i, idx_j] 
    u_matrix[n_kernel, n_kernel] = sum_acc
    
    return u_matrix

def bvector_construction(np.ndarray[DTYPE_t, ndim = 2] reference_image,np.ndarray[DTYPE_t, ndim = 2] data_image,np.ndarray[DTYPE_t, ndim = 2] weights, pandq, n_kernel_np, kernel_size_np):

    cdef int ni_image = np.shape(data_image)[0]
    cdef int nj_image = np.shape(data_image)[1]
    cdef double sum_acc = 0.
    cdef int idx_l,idx_m,idx_l_prime,idx_m_prime,idx_i,idx_j
    cdef int kernel_size = np.int(kernel_size_np)
    cdef int kernel_size_half = np.int(kernel_size_np)/2
    cdef int n_kernel = np.int(n_kernel_np)
        
    cdef np.ndarray b_vector = np.zeros([n_kernel + 1], dtype=DTYPE)
    for idx_p in range(n_kernel):
        idx_l, idx_m = pandq[idx_p]
        sum_acc = 0.
        for idx_i in range(kernel_size_half,ni_image-kernel_size+kernel_size_half+1):
            for idx_j in range(kernel_size_half,nj_image-kernel_size+kernel_size_half+1):
                   sum_acc += data_image[idx_i, idx_j] * reference_image[idx_i + idx_l , idx_j + idx_m ] * weights[idx_i, idx_j]
        b_vector[idx_p] = sum_acc

    sum_acc = 0.
    for idx_i in range(ni_image):
        for idx_j in range(nj_image):
            sum_acc += data_image[idx_i, idx_j] * weights[idx_i, idx_j]
    b_vector[n_kernel] = sum_acc

    return b_vector

In [3]:
# other useful imports
import os
import astropy
from astropy.io import fits
from scipy.signal import convolve2d
from scipy.optimize import minimize
from scipy.stats import norm
from MakeFakeImage import MakeFake # a custom script to generate the images
import time
import torch # PyTorch
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import PyTorchDIA_CCD # PyTorchDIA implementation with a gaussian, CCD noise model
torch.backends.cudnn.deterministic = False

PyTorch version: 1.6.0


In [5]:
## Specify all the functions required ##
## the B08 specific functions are once again adapted from pyDANDIA

# required for B08 approach to deal with edge pixels (convolutions don't like edges!)
def extend_image(image, kernel_size):
    image_extended = np.zeros((np.shape(image)[0] + 2 * kernel_size,
                             np.shape(image)[1] + 2 * kernel_size))
    image_extended[kernel_size:-kernel_size, kernel_size:-kernel_size] = np.array(image, float)
    
    return image_extended


def extend_image_hw(image, kernel_size):
    image_extended = np.zeros((np.shape(image)[0] + kernel_size - 1,
                             np.shape(image)[1] + kernel_size - 1))
    hwidth = np.int((kernel_size - 1) / 2)
    image_extended[hwidth:image_extended.shape[0]-hwidth,
                   hwidth:image_extended.shape[1]-hwidth] = np.array(image, float)
    return image_extended

# add gaussian noise to image under the standard CCD noise model
# N.B. Gain and flat-field are equal to 1, so we only include
# the readout noise [ADU] and the photon shot noise (in the gaussian limit)
def add_noise_to_image(image, read_noise):
    noise_map = np.random.normal(0, 1, size=image.shape)
    sigma_imag = np.sqrt(read_noise**2 + image)
    image += noise_map*sigma_imag
    return image, sigma_imag

# adds 10 times **less** variance than add_noise_to_image
def add_less_noise_to_image(image, read_noise):
    noise_map = np.random.normal(0, 1, size=image.shape)
    sigma_imag = 10**(-0.5) * np.sqrt(read_noise**2 + image)
    image += noise_map*sigma_imag
    return image, sigma_imag

# function to build the kernel, U matrix and b vector
def construct_kernel_and_matrices(kernel_size, R, I, weights):

    pandq = []
    n_kernel = kernel_size * kernel_size
    ncount = 0
    half_kernel_size = int(int(kernel_size) / 2)
    for lidx in range(kernel_size):
        for midx in range(kernel_size):
            pandq.append((lidx - half_kernel_size, midx - half_kernel_size))


    R = R.astype('float64')
    I =  I.astype('float64')
    weights = weights.astype('float64')

    start_time = time.time()
    U = umatrix_construction(R, weights, pandq, n_kernel, kernel_size)
    b = bvector_construction(R, I, weights, pandq, n_kernel, kernel_size)
    print("--- Finished U and b construction in %s seconds ---" % (time.time() - start_time))
    return U, b


# returns the ML least-squares solution for the B08 approach
def lstsq_solution(R, I, U, b, kernel_size):
    
    lstsq_result = np.linalg.lstsq(np.array(U), np.array(b), rcond=None)
    a_vector = lstsq_result[0]
    lstsq_fit = np.dot(np.array(U), a_vector)
    resid = np.array(b) - lstsq_fit
    reduced_chisqr = np.sum(resid ** 2) / (float(kernel_size * kernel_size))
    lstsq_cov = np.dot(np.array(U).T, np.array(U)) * reduced_chisqr
    resivar = np.var(resid, ddof=0) * float(len(a_vector))
    
    # use pinv in order to stabilize calculation
    a_var = np.diag(np.linalg.pinv(lstsq_cov) * resivar)

    a_vector_err = np.sqrt(a_var)
    output_kernel = np.zeros(kernel_size * kernel_size, dtype=float)
    if len(a_vector) > kernel_size * kernel_size:
        output_kernel = a_vector[:-1]
    else:
        output_kernel = a_vector
    output_kernel = output_kernel.reshape((kernel_size, kernel_size))

    err_kernel = np.zeros(kernel_size * kernel_size, dtype=float)
    if len(a_vector) > kernel_size * kernel_size:
        err_kernel = a_vector_err[:-1]
        err_kernel = err_kernel.reshape((kernel_size, kernel_size))
    else:
        err_kernel = a_vector_err
        err_kernel = err_kernel.reshape((kernel_size, kernel_size))

    output_kernel_2 = np.flip(np.flip(output_kernel, 0), 1)
    err_kernel_2 = np.flip(np.flip(err_kernel, 0), 1)
    bkg_kernel = a_vector[-1]
    output_kernel_2.shape

    return output_kernel_2, bkg_kernel

# simply returns the model image
def model_image(R, kernel, B0):
    model = convolve2d(R, kernel, mode='same') + B0
    return model

# return the mean-squared-error (MSE) fit quality metric
def calc_MSE(M, I_noiseless, kernel_size):
    N_data = len(I_noiseless.flatten())
    MSE = 1./(N_data) * np.sum((M - I_noiseless)**2)
    return MSE

# returns the mean-fit-bias (MFB) and mean-fit-variance (MFV) fit quality metrics
def calc_MFB_and_MFV(M, I, noise_map, kernel_size):
    N_data = len(I.flatten())
    MFB = 1./(N_data) * np.sum((I - M)/noise_map)
    MFV = 1./(N_data - 1) * np.sum((((I - M)/noise_map) - MFB)**2)
    return MFB, MFV

# plots the normalised residuals (epsilon) in D overlain with a unit gaussian
def plot_normalised_residuals(epsilon):
    plt.figure(figsize=(5,5))
    plt.hist(epsilon.flatten(), bins='auto', density=True)
    x = np.linspace(-5, 5, 100)
    plt.plot(x, norm.pdf(x, 0, 1))
    plt.xlim(-5, 5)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.xlabel('Normalised residuals', fontsize=20)
    plt.ylabel('Probability', fontsize=20)
    plt.show();

# plot a 2D numpy array    
def plot_image(image, title):
    plt.figure()
    plt.title(title)
    plt.imshow(image)
    plt.colorbar()
    plt.show();

# returns the normalised_psf_object for the PSF fitting photometry
def normalised_psf_object(psf_sigma, psf_size, shifts):
    # first of, let's create the normalised PSF object to fit
    psf = np.zeros((psf_size, psf_size))
    nx, ny = psf_size, psf_size
    xg, yg = np.meshgrid(range(nx), range(ny))
    ## Let's extend this to 2 dimensions ##
    centre = np.int(nx/2)
    #pos = [centre, centre]
    pos = [centre + shifts[0], centre + shifts[1]]
    kernel = np.exp(-0.5 * ((xg - pos[0]) ** 2 + (yg - pos[1]) ** 2)/ psf_sigma ** 2)
    kernel /= np.sum(kernel) # normalise
    psf += kernel #/ (2. * np.pi * psf_sigma ** 2)
    return psf

# cutout around the central, brightest star
def cutout(image, c_size):
    centre = np.int(image.shape[0]/2)
    radius = np.int((c_size/2))
    cutout = image[centre - radius:centre + radius + 1, centre - radius:centre + radius + 1]
    
    # Generate circular mask
    mask_stamp = np.random.normal(0, 3, size=(c_size, c_size))
    for row in mask_stamp:
        for pixel in row:
            coords = np.where(mask_stamp == pixel)
            y_coord, x_coord = coords[0][0], coords[1][0]
            centre_x, centre_y = np.int(cutout.shape[0]/2), np.int(cutout.shape[0]/2)
            delta_x = x_coord - centre_x
            delta_y = y_coord - centre_y
            distance = np.sqrt(delta_x**2 + delta_y**2)
            if distance < radius-1:
                mask_stamp[coords] = 0
            else:
                mask_stamp[coords] = 1
    
    # mask cutout
    #cutout = np.ma.array(cutout, mask=mask_stamp)
    
    return cutout

# results the PSF fitting photometry
def fit_results_and_resids(fit, psf, data):
    F_pred, const_pred = fit.x[0], fit.x[1]
    covmatrix = fit.hess_inv
    var = np.diag(covmatrix)
    print('\nPSF fit results:')
    print('F_pred:', F_pred, np.sqrt(var[0]))
    print('Additive constant:', const_pred, np.sqrt(var[1]))
    message = fit.message
    return F_pred, const_pred, message

# save a np.ndarray as .fits
def save_numpy_as_fits(numpy_array, filename):
    hdu = fits.PrimaryHDU(numpy_array)
    hdul = fits.HDUList([hdu])
    hdul.writeto(filename, overwrite=True)
    
# compute log-likelihood of data given the (MLE) model
def evaluate_log_likelihood(data, model, var):
    chi2 = (data - model)**2 / var
    ln_sigma = np.log(var)
    norm_constant = (len(data.flatten()) / 2) * np.log(2 * np.pi)
    return -(0.5*chi2.sum() + ln_sigma.sum() + norm_constant)

In [8]:
# number of simulations to run
n_simulations = 50000

# number of iterations to perform for each simulation
# for the first three iterations, we update the noise model
# used by the pyDANDIA solution. On the last i.e. 4th iteration
# we call the PyTorch code, which iterates 3 times automatically
n_iters = 4 # 4 normally!

for simulation in range(0, n_simulations):
    
    #np.random.seed(42)
    
    pyDANDIA, PyTorch = False, False
    print('\n\nSimulation:', simulation)
    # generate noiseless reference 'ref'
    print('Generating reference...')
    size = 142
    log_density = np.random.uniform(0, 3, 1)[0]
    star_density = 10**log_density # stars per 100x100 pixels
    n_sources = np.int(star_density * (size**2/100**2))       
    phi_r = np.random.uniform(0.5, 2.5, 1)[0] # in pixels, this is ~ [1 - 6] fwhm
    sky = np.random.uniform(10, 1000, 1)[0] # ADU
    
    # positions
    positions_x = np.random.uniform(0, size, (n_sources,1))
    positions_y = np.random.uniform(0, size, (n_sources,1))
    positions = np.hstack((positions_x, positions_y))

    # fluxes
    F = np.random.uniform(10**(-9), 10**(-4.5), n_sources)
    fluxes = F**(-2./3.)
    
    # Generate the noiseless reference image
    ref_noiseless, F_frac = MakeFake(N=1, size=size, n_sources=n_sources,
                                     psf_sigma=phi_r, sky=sky,
                                     positions=positions, fluxes=fluxes, shifts=[0, 0])
    
    print('Reference properties')
    print('Reference size:', size)
    print('Number of sources:', n_sources)
    print('PSF standard deviation:', phi_r)
    print('Sky level:', sky)
    print('F_max/F_total:', F_frac) # flux ratio of brightest star that of all stars
    
    print('Target kernel properties:')
    phi_k = np.random.uniform(0.5, 2.5, 1)[0]
    kernel_size = 19
    kernel_size = (np.ceil(kernel_size) // 2 * 2 + 1).astype(int) # round up to nearest odd integer
    print('Kernel standard deviation:', phi_k)
    print('Kernel size:', kernel_size)

    # Generate the noiseless (and shifted) target image
    phi_i = np.sqrt(phi_k**2 + phi_r**2)
    
    # positions
    shift_x = np.random.uniform(-0.5, 0.5, 1)
    shift_y = np.random.uniform(-0.5, 0.5, 1)
    #shift_x, shift_y = 0, 0
    print('shift_x, shift_y:', shift_x, shift_y)
    positions_x_shifted = positions_x + shift_x
    positions_y_shifted = positions_y + shift_y
    positions_shifted = np.hstack((positions_x_shifted, positions_y_shifted))

    imag_noiseless, F_frac = MakeFake(N=1, size=size, n_sources=n_sources,
                                     psf_sigma=phi_i, sky=sky,
                                     positions=positions_shifted, fluxes=fluxes,
                                     shifts = [shift_x[0], shift_y[0]])    
    

    imag_noiseless_copy = np.copy(imag_noiseless)
    ref_noiseless_copy = np.copy(ref_noiseless)
    
    # read noise [ADU]
    sigma_0 = 5.
    
    # add noise to the reference image
    ref, sigma_ref = add_less_noise_to_image(image=ref_noiseless, read_noise=sigma_0)
        
    # add noise to the target image i.e.
    imag, sigma_imag = add_noise_to_image(image=imag_noiseless, read_noise=sigma_0) # need noise map for PSF fitting later on
    
    print('Reference and target image shapes:')
    print(ref.shape, imag.shape)
    
    # calculate SNR of images
    SNR_ref = np.sum(ref_noiseless_copy - sky) / np.sqrt(np.sum(sigma_ref**2))
    SNR_imag = np.sum(imag_noiseless_copy - sky) / np.sqrt(np.sum(sigma_imag**2))
    print('Reference SNR:', SNR_ref)
    print('Target SNR:', SNR_imag)
    
    # plot the image pair
    '''
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(ref)
    axarr[0].set_title('Reference Image')
    axarr[1].imshow(imag)
    axarr[1].set_title('Target Image')
    plt.show();
    '''
    
    # exit if SNR_imag > 1000
    if SNR_imag > 1000:
        print('Target SNR regime out of bounds! Skipping...')
        continue
    
    # 'sky' subtract reference... **crucial** for getting past
    # the strong anticorrelation between P and B0
    print("Sky subtracting reference.")
    #ref -= np.median(ref)
    ref -= sky

    for i in range(0, n_iters):
               
        if i == 0:
            
        # for first pass, estimate weights with inverse variance map
            weights = 1./(imag + sigma_0**2)
        else:
            weights = 1./(M + sigma_0**2)
            
        if i < 3:
            
            print('\nB08 solution, iteration %d/%d' % (i+1, 3))
            
            '''
            # extend boundaries of images for B08 solution
            ext_ref = extend_image(ref, kernel_size)
            ext_imag = extend_image(imag, kernel_size)
            ext_weights = extend_image(weights, kernel_size)

            U, b = construct_kernel_and_matrices(kernel_size, ext_ref, ext_imag, ext_weights)
            kernel, B0 = lstsq_solution(ext_ref, ext_imag, U, b, kernel_size)
            '''
            
            # extend boundaries of images for B08 solution
            ext_ref = extend_image_hw(ref, kernel_size)
            ext_imag = extend_image_hw(imag, kernel_size)
            ext_weights = extend_image_hw(weights, kernel_size)

            U, b = construct_kernel_and_matrices(kernel_size, ext_ref, ext_imag, ext_weights)
            kernel, B0 = lstsq_solution(ext_ref, ext_imag, U, b, kernel_size)            
            
            if i == 2:
                # compute fit quality and photometric accuracy metrics on this iteration
                pyDANDIA = True
                #plt.imshow(kernel)
                #plt.colorbar();
                #plt.title('B08 kernel')
                #plt.show();
        
            
        elif i == 3:
            print('\nPyTorchDIA solution')
            pyDANDIA = False
            PyTorch = True
            SD_steps = 25000
            
            kernel, B0 = PyTorchDIA_CCD.DIA(ref,
                                       imag,
                                       np.ones(imag.shape), # flatfield
                                       rdnoise=sigma_0,
                                       G = 1,
                                       ks = kernel_size,
                                       lr_kernel = 1e-3,
                                       lr_B = 10,
                                       max_iterations = 25000,
                                       poly_degree=0,
                                       alpha = 0.,
                                       Newton_tol = 1e-6,
                                       tol = 1e-9,
                                       fast=True,
                                       fisher=False,
                                       show_convergence_plots=False)
            
            
            #plt.imshow(kernel)
            #plt.show()
            
            # avoid nans
            if np.any(np.isnan(kernel)) is True:
                print('NaNs in kernel')
                continue

            #if np.isnan(np.sum(kernel)) == True:
            #    continue
      
     
        ## compute model image ##
        ## N.B. we extend the border of ref to handle edge-effects associated with the convolution
        '''
        ext_ref = extend_image(ref, kernel_size)
        ext_M = model_image(ext_ref, kernel, B0)
        M = ext_M[kernel_size:ext_M.shape[0]-kernel_size, kernel_size:ext_M.shape[1]-kernel_size]
        '''
        ext_ref = extend_image_hw(ref, kernel_size)
        ext_M = model_image(ext_ref, kernel, B0)
        hwidth = np.int((kernel_size - 1) / 2)
        M = ext_M[hwidth:ext_M.shape[0]-hwidth, hwidth:ext_M.shape[1]-hwidth]        

        if np.any(M < 0.) is True:
            print('Negatives in Model Image! Fitting failed!')
            print('Skipping simulation...')
            break
        
        ## on the final iteration only, compute fit metrics,
        ## and perform PSF fitting photometry
        ## at the position of the brightest star
        if pyDANDIA == True or PyTorch == True:
            
            # print best fit parameters
            print('\nPhotometric Scale Factor:', np.sum(kernel))
            print('B_0:', B0)


            # Fit quality metrics
            MSE = calc_MSE(M, imag_noiseless, kernel_size)
            MFB, MFV = calc_MFB_and_MFV(M, imag, sigma_imag, kernel_size)
            #MSEs, MFBs, MFVs = np.append(MSEs, MSE), np.append(MFBs, MFB), np.append(MFVs, MFV)
            print('\nQuality Metrics:')
            print('MSE', MSE)
            print('MFB', MFB)
            print('MFV', MFV)
            print('\n')

            # compute the difference image / fit residuals and inferred pixel_variances
            D = imag - M
            
            
            # inspect normalised residuals against a unit gaussian
            pixel_variances = sigma_0**2 + M # G=1, F_ij=1
            
            '''
            norm_resids = D / np.sqrt(pixel_variances)
            plt.figure(figsize=(5,5))
            plt.hist(norm_resids.flatten(), bins='auto', density=True)
            x = np.linspace(-5, 5, 100)
            plt.plot(x, norm.pdf(x, 0, 1))
            plt.xlim(-5, 5)
            plt.xticks(fontsize=20)
            plt.yticks(fontsize=20)
            plt.xlabel('Normalised residuals', fontsize=20)
            plt.ylabel('Probability', fontsize=20)
            plt.show();
            '''
            
            # evaluate the log-likelihood of the data under the (MLE) model
            ll = evaluate_log_likelihood(imag, M, pixel_variances)
            print('Log-likelihood:', ll)
            
            ## PSF fitting photometry of brightest, central star ##
            stamp_size = phi_i * 9 # i.e. about 4*target_FWHM
            stamp_size = (np.ceil(stamp_size) // 2 * 2 + 1).astype(int) # round up to nearest odd integer
            
            psf_object = normalised_psf_object(phi_r, stamp_size, [0., 0.])
            true_target_psf_object = normalised_psf_object(phi_i, stamp_size, [shift_x[0], shift_y[0]])

            # convolve normalised psf_object with the kernel and re-normalise
            # remember to deal with the borders appropriately!
            ext_psf_object = extend_image(psf_object, kernel_size)
            ext_target_psf_object = convolve2d(ext_psf_object, kernel, mode='same')
            target_psf_object = ext_target_psf_object[kernel_size:ext_target_psf_object.shape[0]-kernel_size,
                                                      kernel_size:ext_target_psf_object.shape[1]-kernel_size]
            
            target_psf_object = target_psf_object / np.sum(target_psf_object)
            
            # make stamp around position of bright star in D
            stamp = cutout(D, stamp_size)

            # cutout target image pixel noise to weight the fit
            noise_stamp = cutout(sigma_imag, stamp_size)

            # initialise fit parameters: the difference flux and additive constant
            F_diff = torch.nn.Parameter(torch.ones(1), requires_grad=True)
            const = torch.nn.Parameter(torch.ones(1), requires_grad=True)

            # convert numpy.ndarray to torch.Tensor
            target_psf_object = torch.from_numpy(target_psf_object)
            stamp = torch.from_numpy(stamp)
            noise_stamp = torch.from_numpy(noise_stamp)
            
            # chi-square likelihood, as pixel variances known a priori
            class log_likelihood(torch.nn.Module):
                def forward(model, stamp, noise_stamp):
                    loglikelihood = -0.5*(((stamp - model)/noise_stamp)**2).sum()
                    return -loglikelihood
            
            # initialise optimizer
            optimizer = torch.optim.Adam([F_diff, const], lr=10)

            tol = 1e-9
            losses = []
            F_diffs = []
            
            # optimally scale F_diff to the target_psf_object
            for i in range(0, 1000000):
                optimizer.zero_grad()
                model = F_diff*target_psf_object + const
                F_diffs.append(F_diff.item())
                loss = log_likelihood.forward(model, stamp, noise_stamp)
                losses.append(loss.item())
                loss.backward()
                optimizer.step()
                
                if i>1 and abs((losses[-1] - losses[-2])/losses[-2]) < tol:
                    print('Converged')
                    break
                

            print('Fitted F_diff and const:', F_diff, const)
             
            ## convert tensors back to numpy arrays
            F_diff = F_diff.detach().numpy()
            const = const.detach().numpy()
            target_psf_object = target_psf_object.detach().numpy()
            stamp = stamp.detach().numpy()
            noise_stamp = noise_stamp.detach().numpy()
            
            ## compute normalised residuals
            prediction = F_diff*target_psf_object + const
            residuals_stamp = (prediction - stamp) / noise_stamp
            
            ## inspect images as .fits
            #save_numpy_as_fits(D/np.sqrt(pixel_variances), 'residuals_D.fits')
            #save_numpy_as_fits(stamp, 'stamp.fits')
            #save_numpy_as_fits(cutout(imag, stamp_size), 'imag_stamp.fits')
            #save_numpy_as_fits(target_psf_object, 'target_psf_object.fits')
            #save_numpy_as_fits(true_target_psf_object, 'true_target_psf_object.fits')
            #save_numpy_as_fits(residuals_stamp, 'residuals_stamp.fits')
            

            ## Compute F_measured of brightest star
            F_measured = F_diff / np.sum(kernel)

            # Compute the theoretical minimum variance for F_measured
            P_true = 1.
            var_min = (1./P_true**2) * (np.sum((true_target_psf_object**2)/(noise_stamp**2)))**(-1)
            print('F_measured/sigma_min:', F_measured/np.sqrt(var_min))

            out = np.vstack((np.sum(kernel), B0, MSE, MFB, MFV, F_measured, var_min,
                            star_density, phi_r, sky, phi_k,
                            SNR_ref, SNR_imag, F_frac, shift_x, shift_y, ll))
            
            #path = '/media/james/Seagate_Expansion_Drive#2'
            path = os.getcwd()
            
            if pyDANDIA == True:
                filename = os.path.join(path, 'pyDANDIA_December2020_JI.txt')
                with open(filename, 'a') as f:
                    np.savetxt(f, out.T)
            elif PyTorch == True:
                filename = os.path.join(path, 'PyTorch_December2020_JI.txt')
                with open(filename, 'a') as f:
                    np.savetxt(f, out.T)            
            




Simulation: 0
Generating reference...
Max flux: 13670.299078749502
Frac for 142x142 image: 0.3353160435001864
Reference properties
Reference size: 142
Number of sources: 17
PSF standard deviation: 2.296175704929089
Sky level: 915.288635324258
F_max/F_total: 0.3353160435001864
Target kernel properties:
Kernel standard deviation: 0.5064648501726285
Kernel size: 19
shift_x, shift_y: [0.45491893] [-0.10441395]
Max flux: 13670.299078749502
Frac for 142x142 image: 0.3353160435001864
Reference and target image shapes:
(142, 142) (142, 142)
Reference SNR: 29.575924498442177
Target SNR: 9.35272853202504
Sky subtracting reference.

B08 solution, iteration 1/3


KeyboardInterrupt: 